# Have to rerun everything when i have more subjects - as the group ICA needs to be re-estimated and then all that comes afterwards (which is essentially everything)

For future self:
* don't use the _Z files in .dr folders. Less recommended in my understanding. 

* Do subgroups analysis


* to install packages and maybe run some stuff in the terminal I might need sometimes to run: conda activate myenv


# Import stuff + params

In [1]:
import os
import glob
import numpy as np
import pandas as pd

# need to set up to run fsl.
os.environ['FSLDIR'] = '/export/home/ranigera/fsl'
os.environ['PATH'] += ':/export/home/ranigera/fsl/bin'

group_ICA_path = '/export/home/ranigera/IDCH-rsfMRI/data/group_ICA'
rs_data_path = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data'
rs_data_path_IDCH_sub_Nums = '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums'

input_files_file_name = 'input_files.txt'
input_files_txt_path = os.path.join(group_ICA_path, input_files_file_name)
n_ICs = 50 # number of independent components
TR = 0.70 # repetition time in seconds

# (original) file_format:
any_session_file_format = '_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz'
stage2_ic_file_prefix = 'dr_stage2_ic' # move to params

# R01 to IDCH mapping:
mapping_R01_to_IDCH = {'222': '101', '183': '102', '216': '103', '192': '104', '251': '105', '206': '106', '180': '107', '184': '108', '169': '109', '207': '110',
                       '159': '111', '115': '112', '114': '113', '232': '114', '173': '115', '171': '117', '215': '118', '265': '119', '177': '120', '269': '121',
                       '261': '122'}

main_behav_file = '/export/home/ranigera/IDCH-rsfMRI/data/behav_data/merged_behav_data.csv'

# Preperation

## Copy data to be with IDCH sub numbers (and extract the brain)

In [2]:
# get all directories in the rs_data_path:
r01_sub_dirs = [f for f in os.listdir(rs_data_path) if os.path.isdir(os.path.join(rs_data_path, f))]
# iterate over all subjects:
for dir in r01_sub_dirs:
    sub_dir = os.path.join(rs_data_path, dir, 'func') 
    sub_R01_id = dir.split('Hab0')[1][:3] # get sub R01 id
    sub_IDCH_id = mapping_R01_to_IDCH[sub_R01_id] # get sub IDCH id

    # get subject's resting state:
    rs_files = [f for f in os.listdir(sub_dir) if any_session_file_format in f]
    for f in rs_files:
        # get the session number:
        session = f.split('rest_run-')[1][0]
        new_file_name = f'sub-{sub_IDCH_id}_ses-{session}_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz'
        # copy the file if not exists:
        if not os.path.exists(os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)):
            print(f'> Copying {f} to {new_file_name}')
            # os.system(f'cp {os.path.join(sub_dir, f)} {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
            os.system(f'fslmaths {os.path.join(sub_dir, f)} -mas $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz {os.path.join(rs_data_path_IDCH_sub_Nums, new_file_name)}')
        # write the file name to the input_files.txt file:


## Create input_files.txt file needed for the group ICA

In [3]:
# create group_ICA_path if it does not exist
if not os.path.exists(group_ICA_path):
    os.makedirs(group_ICA_path)

# list all files in all subdirectories of rs_data_path_IDCH_sub_Nums
rs_data_files = []
for root, dirs, files in os.walk(rs_data_path_IDCH_sub_Nums):
    for file in files:
        if file.endswith('.nii.gz'):
            rs_data_files.append(os.path.join(root, file))
print('Number of files:', len(rs_data_files))
rs_data_files.sort()

with open(input_files_txt_path, 'w') as f:
    for file in rs_data_files:
        f.write(file + '\n')
rs_data_files

Number of files: 42


['/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-104_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz',
 '/export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/s

# Run group ICA (creates commad)

In [4]:
n_ICs=50 # number of independent components
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
n_ICs=40
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
n_ICs=30
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
n_ICs=20
print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{n_ICs} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d {n_ICs}')
# Determine # of components automatically (tend to create a LOT of components):
# print(f'melodic -i {input_files_txt_path} -o {group_ICA_path}/groupICA{"auto_n_ICs"} --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall')


melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA50 --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d 50
melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA40 --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d 40
melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA30 --tr=0.70 --nobet -a concat --bgimage=$FSLDIR/data/standard/MNI152_T1_2mm_brain.nii.gz -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz --report --Oall -d 30
melodic -i /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/input_files.txt -o /expor

# Run dual regression (creates command)

* This is dual regression without the 3rd step of group analysis. This is done separately later)

In [5]:
n_ICs=20 # number of independent components
print(f'dual_regression {group_ICA_path}/groupICA{n_ICs}/melodic_IC 1 \
      -1 0 \
      {group_ICA_path}/groupICA{n_ICs}.dr {" ".join(item for item in rs_data_files)}')

dual_regression /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20/melodic_IC 1       -1 0       /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.dr /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-101_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-102_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-1_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH_subNums/sub-103_ses-2_rest_MNI152-smoothAROMAnonaggr_bold_brain.nii.gz /export/home/ranigera/IDCH-rsfMRI/data/pre_proc_data_IDCH

## Averaging each subect's data (over the two sessions) for each IC [and renaming back to IDCH sub-IDs instead of the one attached automaticaaly starting at 0000]

In [6]:
n_ICs=20

maching_IDCH_sub_list = sorted(mapping_R01_to_IDCH.values())
sub_avg_folder = os.path.join(group_ICA_path, f'groupICA{n_ICs}.sub_avg')
if not os.path.exists(sub_avg_folder):
    os.makedirs(sub_avg_folder)

for IC in range(n_ICs):
    print(f'Processing IC {IC}')
    IC_str = str(IC).zfill(4) # get the IC into a string that has 4 digits

    # Check that files were not already created:
    sub_IC_avg_files = [f for f in os.listdir(sub_avg_folder) if f.startswith(f'IC{IC_str}_sessions-avg_ordered-sub-')]
    N_sub_by_ses_files = [f for f in os.listdir(rs_data_path_IDCH_sub_Nums)]    
    if len(sub_IC_avg_files) == len(N_sub_by_ses_files)/2:
        print(f'All IC{IC_str}_sessions-avg_ordered-sub-*.nii.gz already exists')
        continue
    
    # split the IC data into sub-ICs:
    file_name = f'{stage2_ic_file_prefix}{IC_str}.nii.gz'
    file_path = os.path.join(group_ICA_path, f'groupICA{n_ICs}.dr', file_name)
    print('Splitting:', file_path)
    os.system(f'fslsplit {file_path} {sub_avg_folder}/IC{IC_str}_ -t') # split the IC into sub-ICs:

    # get the sub_IC_files:
    sub_IC_files = [f for f in os.listdir(sub_avg_folder) if f.startswith(f'IC{IC_str}_')]
    sub_IC_files = [f for f in sub_IC_files if 'avg' not in f]
    sub_IC_files.sort()

    # iterate over the sub_IC_files in pairs and concatenate them:
    for i in range(0, len(sub_IC_files), 2):
        sub_file1 = os.path.join(sub_avg_folder, sub_IC_files[i])
        sub_file2 = os.path.join(sub_avg_folder, sub_IC_files[i+1])
        # check if the concatenated file exists:
        if not os.path.exists(f'{sub_avg_folder}/IC{IC_str}_sessions-avg_ordered-sub-{maching_IDCH_sub_list[i//2]}.nii.gz'):
            os.system(f'fslmaths {sub_file1} -add {sub_file2} -div 2 {sub_avg_folder}/IC{IC_str}_sessions-avg_ordered-sub-{maching_IDCH_sub_list[i//2]}.nii.gz') # this calculate the average in the right way. I TESTED IT!
            print('processing:', sub_file1, sub_file2)
            print(f'Created {sub_avg_folder}/IC{IC_str}_sessions-avg_ordered-sub-{maching_IDCH_sub_list[i//2]}.nii.gz')
        else:
            print(f'IC{IC_str}_sessions-avg_ordered-sub-{maching_IDCH_sub_list[i//2]}.nii.gz already exists')

    # remove the sub_IC_files:
    for f in sub_IC_files:
        os.system(f'rm {os.path.join(sub_avg_folder, f)}')


Processing IC 0
All IC0000_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 1
All IC0001_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 2
All IC0002_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 3
All IC0003_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 4
All IC0004_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 5
All IC0005_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 6
All IC0006_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 7
All IC0007_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 8
All IC0008_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 9
All IC0009_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 10
All IC0010_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 11
All IC0011_sessions-avg_ordered-sub-*.nii.gz already exists
Processing IC 12
All IC0012_sessions-avg_ordered-sub-*.nii.gz already exists
Processin

# Test correlation with behavior (using randomise)

## Get behavior data

In [9]:
# Initial general stuff:

IC_behavior_corr_folder = os.path.join(group_ICA_path, f'groupICA{n_ICs}.behavior_corr')
if not os.path.exists(IC_behavior_corr_folder):
    os.makedirs(IC_behavior_corr_folder)

# load the behavior data:
main_behav_data = pd.read_csv(main_behav_file)
# change subID to int:
main_behav_data['subID'] = main_behav_data['subID'].astype(int)
main_behav_data

,subID,sequenceCompleted_test,sequenceCompleted_reacquisition,sequenceCompleted_combined,at_least_one_response_test,at_least_one_response_reacquisition,at_least_one_response_combined,devaluation,stillVal_minus_deval,preVal_relativeDiff_deval,meanVal_minus_deval,meanVal_relativeDiff_deval,meanVal_relativeDiff_deval_SQRT,std_diary,normed_std_diary,SRM_score,mood,Anxiety,Stress,routine_mean
0,101,1,11,12,1,11,12,0,34,1.000000,34.5,1.000000,1.000000,1281.631619,0.530220,2.200000,71.985294,34.367647,26.191176,3.593137
1,102,0,0,0,0,0,0,0,5,1.000000,4.0,1.000000,1.000000,966.548720,0.433184,4.125000,68.357143,22.000000,21.828571,3.638095
2,103,0,0,0,0,0,0,6,1,0.076923,-1.0,-0.090909,-0.045549,761.555304,0.372977,6.000000,63.100000,20.471429,18.957143,3.823810
3,104,0,0,0,0,0,0,3,12,0.666667,6.5,0.520000,0.280449,1200.262012,0.519515,3.058824,68.457143,14.714286,14.814286,4.095238
4,105,3,0,3,4,0,4,0,9,1.000000,8.0,1.000000,1.000000,1444.988375,0.595996,1.882353,62.536232,33.927536,42.318841,4.376812
5,106,1,0,1,2,0,2,4,26,0.764706,11.0,0.578947,0.318915,NaN,NaN,NaN,90.724138,3.896552,9.327586,5.902299
6,107,0,0,0,0,0,0,0,10,1.000000,6.5,1.000000,1.000000,1376.074428,0.555342,1.954545,66.402985,42.507463,37.104478,4.457711
7,108,0,0,0,0,0,0,0,3,1.000000,2.5,1.000000,1.000000,1065.073426,0.452931,1.571429,70.609375,25.718750,15.906250,5.072917
8,109,0,0,0,0,0,0,2,25,0.862069,15.5,0.794872,0.494699,971.316793,0.462965,3.000000,86.630769,3.384615,5.400000,5.907692
9,110,0,0,0,1,0,1,7,7,0.333333,2.0,0.125000,0.062746,1014.818277,0.480114,4.600000,66.428571,12.442857,28.271429,5.219048


## Run the test using these steps:
1. Concatenating the files across subjects per IC
(Done after the session were averaged, as a preliminary step for randomise)
2. Create design matrix and con file
3. Run randomise
4. Show the min/max values of the results (1 - p_value). (Therefore > 0.95 is significant)

In [10]:
def get_IC_behav_corr_folder(n_ICs=20, behav_var='SRM_score'):
    # ----------------------------------------------------------------------
    # STEP 1: Create the nii files for the dual-regression analysis:
    # ----------------------------------------------------------------------
    # Rewrite if exists:
    rewrite = input(f"Do you want to recreate the nii combined files [for the following dual-regression analysis] (if exist)? (yes/[No]): ")

    # get the data with subID and the behavior variable (and remove NaNs):
    behav_data = main_behav_data[['subID', behav_var]].dropna().reset_index(drop=True)

    # create a directory:
    IC_behav_corr_folder = os.path.join(IC_behavior_corr_folder, f'IC_behav_corr_{behav_var}')
    IC_behav_corr_input_data_folder = os.path.join(IC_behav_corr_folder, 'input_data')
    if not os.path.exists(IC_behav_corr_input_data_folder):
        os.makedirs(IC_behav_corr_input_data_folder)

    for IC in range(n_ICs):
        print(f'Concatenating subjects of IC ** {IC} ** (after averaging the sessions)')
        IC_str = str(IC).zfill(4) # get the IC into a string that has 4 digits

        sub_avg_files = glob.glob(f'{sub_avg_folder}/IC{IC_str}_sessions-avg_ordered-sub-*.nii.gz')
        sub_avg_files = [f for f in sub_avg_files if int(f.split('sub-')[1].split('.')[0]) in behav_data['subID'].values] # removes the files that are not in the behav_data
        sub_avg_files.sort()
        
        # file to create:
        file_name = f'{stage2_ic_file_prefix}{IC_str}_sub-sessions-avg_for-{behav_var}.nii.gz'
        file_path = os.path.join(IC_behav_corr_input_data_folder, file_name)

        # concatenate subject files to recreate the IC file with all subjects (after averaging the sessions):
        print('Concatenating to form:', file_path)
        if os.path.exists(file_path) and rewrite.lower() != 'yes':
            print(f'{file_path} already exists')
            continue

        # print(f'fslmerge -t {file_path} {" ".join(item for item in sub_avg_files)}') # split the IC into sub-ICs:
        os.system(f'fslmerge -t {file_path} {" ".join(item for item in sub_avg_files)}') # split the IC into sub-ICs:


    # ----------------------------------------------------------------------
    # STEP 2: Create the design matrix and contrast files for the IC-behavior correlation analysis:
    # ----------------------------------------------------------------------
    num_data_points = len(behav_data['subID'])
    demeaned_behav_data = behav_data[behav_var] - behav_data[behav_var].mean()

    # Prepare the .mat file content
    # ------------------------------
    header = f"""/NumWaves\t1
    /NumPoints\t{num_data_points}

    /Matrix
    """
    # Format the matrix content
    matrix_content = "\n".join([f"{value}\t" for value in demeaned_behav_data])
    # Combine header and matrix content
    design_matrix_content = header + matrix_content
    print(design_matrix_content)
    # Write the design matrix to a file
    output_file = f'design_IC_behav_corr_{behav_var}.mat'
    output_file = os.path.join(IC_behav_corr_input_data_folder, output_file)
    with open(output_file, 'w') as file:
        file.write(design_matrix_content)
    print(f"Design matrix file '{output_file}' created successfully!")


    # Prepare the .con file content
    # -----------------------------
    # Parameters for the contrast file
    contrast_name1 = "Pos corr"
    contrast_name2 = "Neg corr"
    num_waves = 1  # As per your new design, only one EV
    num_contrasts = 2  # Two contrasts: positive and negative

    # Matrix for contrasts
    # Positive correlation: 1 for the EV
    # Negative correlation: -1 for the EV
    contrast_matrix = [
        [1.0],  # Positive correlation
        [-1.0]  # Negative correlation
    ]

    # Create the .con file content
    header = f"""/ContrastName1\t{contrast_name1}
    /ContrastName2\t{contrast_name2}
    /NumWaves\t{num_waves}
    /NumContrasts\t{num_contrasts}

    /Matrix
    """
    # Format the matrix content
    matrix_content = "\n".join(["\t".join([f"{value}" for value in row]) for row in contrast_matrix])
    # Combine header and matrix content
    contrast_matrix_content = header + matrix_content
    print(contrast_matrix_content)
    # Write the contrast matrix to a file
    output_file = f'design_IC_behav_corr_{behav_var}.con'
    output_file = os.path.join(IC_behav_corr_input_data_folder, output_file)
    with open(output_file, 'w') as file:
        file.write(contrast_matrix_content)
    print(f"Contrast matrix file '{output_file}' created successfully!")

    # ----------------------------------------------------------------------
    # STEP 3: run randomise:
    # ----------------------------------------------------------------------
    run_randomise = input(f"Do you want to run randomise for the IC-behavior correlation analysis? (yes/[No]): ")
    if run_randomise.lower() == 'yes':
        for IC in range(n_ICs):
            IC_str = str(IC).zfill(4) # get the IC into a string that has 4 digits
            IC_file = f'{stage2_ic_file_prefix}{IC_str}_sub-sessions-avg_for-{behav_var}.nii.gz'
            IC_file_path = os.path.join(IC_behav_corr_input_data_folder, IC_file)
            print(f'Processing IC {IC} with file: {IC_file_path}')
            os.system(f'taskset -c 0-47 randomise_parallel -i {IC_file_path} -o {IC_behav_corr_folder}/IC{IC_str}_corr_{behav_var} -d {IC_behav_corr_input_data_folder}/design_IC_behav_corr_{behav_var}.mat -t {IC_behav_corr_input_data_folder}/design_IC_behav_corr_{behav_var}.con -m $FSLDIR/data/standard/MNI152_T1_2mm_brain_mask.nii.gz -n 5000 -D -T')

    print(f'>> Showing results summary of (1-pvlue) range incorrected tfce maps:')
    os.system(f'for i in  {IC_behav_corr_folder}/*_tfce_corrp_tstat*.nii.gz ; do     echo $i $(fslstats $i -R); done')


In [11]:
get_IC_behav_corr_folder(n_ICs=20, behav_var='SRM_score')

Concatenating subjects of IC ** 0 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_SRM_score/input_data/dr_stage2_ic0000_sub-sessions-avg_for-SRM_score.nii.gz
/export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_SRM_score/input_data/dr_stage2_ic0000_sub-sessions-avg_for-SRM_score.nii.gz already exists
Concatenating subjects of IC ** 1 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_SRM_score/input_data/dr_stage2_ic0001_sub-sessions-avg_for-SRM_score.nii.gz
/export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_SRM_score/input_data/dr_stage2_ic0001_sub-sessions-avg_for-SRM_score.nii.gz already exists
Concatenating subjects of IC ** 2 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/gr

In [21]:
get_IC_behav_corr_folder(n_ICs=20, behav_var='meanVal_relativeDiff_deval')

Concatenating subjects of IC ** 0 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_meanVal_relativeDiff_deval/input_data/dr_stage2_ic0000_sub-sessions-avg_for-meanVal_relativeDiff_deval.nii.gz
/export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_meanVal_relativeDiff_deval/input_data/dr_stage2_ic0000_sub-sessions-avg_for-meanVal_relativeDiff_deval.nii.gz already exists
Concatenating subjects of IC ** 1 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_meanVal_relativeDiff_deval/input_data/dr_stage2_ic0001_sub-sessions-avg_for-meanVal_relativeDiff_deval.nii.gz
/export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_meanVal_relativeDiff_deval/input_data/dr_stage2_ic0001_sub-sessions-avg_for-meanVal_relativeDiff_deval.nii.gz already exi

In [12]:
get_IC_behav_corr_folder(n_ICs=20, behav_var='at_least_one_response_test')

Concatenating subjects of IC ** 0 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_at_least_one_response_test/input_data/dr_stage2_ic0000_sub-sessions-avg_for-at_least_one_response_test.nii.gz
Concatenating subjects of IC ** 1 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_at_least_one_response_test/input_data/dr_stage2_ic0001_sub-sessions-avg_for-at_least_one_response_test.nii.gz
Concatenating subjects of IC ** 2 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/group_ICA/groupICA20.behavior_corr/IC_behav_corr_at_least_one_response_test/input_data/dr_stage2_ic0002_sub-sessions-avg_for-at_least_one_response_test.nii.gz
Concatenating subjects of IC ** 3 ** (after averaging the sessions)
Concatenating to form: /export/home/ranigera/IDCH-rsfMRI/data/gro

# Archive

## Concatenating the data (all participants in order)
* Note that this should be used only if all the behavioral data is used. Otherwise, a dedicated concatenation should be performed

In [ ]:
# n_ICs=20

# sub_avg_concat_folder = os.path.join(group_ICA_path, f'groupICA{n_ICs}.sub_avg_dr')
# if not os.path.exists(sub_avg_concat_folder):
#     os.makedirs(sub_avg_concat_folder)

# for IC in range(n_ICs):
#     print(f'Concatenating subjects of IC ** {IC} ** (after averaging the sessions)')
#     IC_str = str(IC).zfill(4) # get the IC into a string that has 4 digits

#     sub_avg_files = glob.glob(f'{sub_avg_folder}/IC{IC_str}_sessions-avg_ordered-sub-*.nii.gz')
#     sub_avg_files.sort()
    
#     # file to create:
#     file_name = f'{stage2_ic_file_prefix}{IC_str}_sub-sessions-avg.nii.gz'
#     file_path = os.path.join(sub_avg_concat_folder, file_name)

#     # concatenate subject files to recreate the IC file with all subjects (after averaging the sessions):
#     print('Concatenating to form:', file_path)
#     if os.path.exists(file_path):
#         print(f'{file_path} already exists')
#         continue

#     print(f'fslmerge -t {file_path} {" ".join(item for item in sub_avg_files)}') # split the IC into sub-ICs:
#     os.system(f'fslmerge -t {file_path} {" ".join(item for item in sub_avg_files)}') # split the IC into sub-ICs:
